# Tugas 3 Memanggil data iris di 4 database yang berbeda Menggunakan PCA




# GetData


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pandas
!pip install psycopg2
!pip install jdbc
!pip install mysql-connector-python
!pip install orchest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement jdbc (from versions: none)
ERROR: No matching distribution found for jdbc
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 34.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 40.4 MB/s eta 0:00:00
  Created wheel for orchest: filename=orchest-0.3.11-py3-none-any.whl size=19359 sha256=168da472bce461a9e33feb6e57c09270a873e399033f22ff271d8b2f82d10d6d
  Stored in 

In [6]:
import psycopg2
import pandas as pd

# Connection parameters, yours will be different
param_dic = {
    "host"      : "trumpet.db.elephantsql.com",
    "database"  : "xtzkuaqv",
    "user"      : "xtzkuaqv",
    "password"  : "OupjNX9j0ny4IIeAWJpC014w6HPFTA38"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Mengambil data dari cursor
    rows = cursor.fetchall()
    cursor.close()
    
    # Membuat DataFrame dari hasil query
    df = pd.DataFrame(rows, columns=column_names)
    
    return df

# Membuat koneksi database
conn = connect(param_dic)

# Menjalankan query dan menghasilkan DataFrame
query = "SELECT sepal_w FROM iris"
column_names = ["sepal_w"]
df1 = postgresql_to_dataframe(conn, query, column_names)

# Menampilkan data
print(df1)

Connecting to the PostgreSQL database...
Connection successful
     sepal_w
0        3.5
1        3.0
2        3.2
3        3.1
4        3.6
..       ...
145      3.0
146      2.5
147      3.0
148      3.4
149      3.0

[150 rows x 1 columns]


In [7]:
#Mysql local
import mysql.connector as mysql
import pandas as pd

myconn = mysql.connect(host='sql12.freesqldatabase.com',
                       database='sql12604228', 
                       user='sql12604228', 
                       password='FT4AaDU9kD')

# Membuat objek cursor
mycursor = myconn.cursor()

# Mengambil data dari tabel
mycursor.execute("SELECT petallength FROM iris")

# Mengambil semua hasil query
result = mycursor.fetchall()

# Menutup kursor dan koneksi
mycursor.close()
myconn.close()

# Membuat DataFrame dari hasil query
df2 = pd.DataFrame(result, columns=["petallength"])

# Menampilkan data
print(df2)

    petallength
0           1.4
1           1.4
2           1.3
3           1.5
4           1.4
..          ...
145         5.2
146           5
147         5.2
148         5.4
149         5.1

[150 rows x 1 columns]


In [8]:
# membaca file csv
iris_df = pd.read_csv('/content/drive/MyDrive/prosaindata/tugas/tugas-prosaindata/iris.csv')

# melakukan seleksi data pada kolom 'sepallength', 'petalwidth', dan 'class'
iris_selection = iris_df[['sepallength', 'petalwidth', 'class']]

# menampilkan DataFrame hasil seleksi data
print(iris_selection.head())

   sepallength  petalwidth        class
0          5.1         0.2  Iris-setosa
1          4.9         0.2  Iris-setosa
2          4.7         0.2  Iris-setosa
3          4.6         0.2  Iris-setosa
4          5.0         0.2  Iris-setosa


In [9]:
import pandas as pd
import orchest

# Gabungkan data dari beberapa sumber ke dalam satu DataFrame
data = pd.concat([df1, df2, iris_selection], axis=1)
print(data)

     sepal_w petallength  sepallength  petalwidth           class
0        3.5         1.4          5.1         0.2     Iris-setosa
1        3.0         1.4          4.9         0.2     Iris-setosa
2        3.2         1.3          4.7         0.2     Iris-setosa
3        3.1         1.5          4.6         0.2     Iris-setosa
4        3.6         1.4          5.0         0.2     Iris-setosa
..       ...         ...          ...         ...             ...
145      3.0         5.2          6.7         2.3  Iris-virginica
146      2.5           5          6.3         1.9  Iris-virginica
147      3.0         5.2          6.5         2.0  Iris-virginica
148      3.4         5.4          6.2         2.3  Iris-virginica
149      3.0         5.1          5.9         1.8  Iris-virginica

[150 rows x 5 columns]


# Preprocessing


In [10]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

In [11]:
#preprosessing

X=data.iloc[:,0:4].values
y=data.iloc[:,4].values

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

# PCA

In [12]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from google.colab import drive

# define the file path for orchest output
output_path = '/content/drive/MyDrive/prosaindata/tugas/tugas-prosaindata/pca_data.csv'

# define the preprocessing steps
preprocessing_steps = [
    StandardScaler(),
]

# fit and transform the preprocessing steps to X
for step in preprocessing_steps:
    X = step.fit_transform(X)

# define the PCA model with n_components = 2
pca = PCA(n_components=2)

# fit and transform PCA to X
X_pca = pca.fit_transform(X)

# create DataFrame for PCA output
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
pca_df['target'] = y

# output the transformed data to csv file in Google Drive
pca_df.to_csv(output_path, index=False)


In [20]:
# display the DataFrame
print(pca_df)

          PC1       PC2          target
0   -2.264542  0.505704     Iris-setosa
1   -2.086426 -0.655405     Iris-setosa
2   -2.367950 -0.318477     Iris-setosa
3   -2.304197 -0.575368     Iris-setosa
4   -2.388777  0.674767     Iris-setosa
..        ...       ...             ...
145  1.870522  0.382822  Iris-virginica
146  1.558492 -0.905314  Iris-virginica
147  1.520845  0.266795  Iris-virginica
148  1.376391  1.016362  Iris-virginica
149  0.959299 -0.022284  Iris-virginica

[150 rows x 3 columns]


# Metode Knn

In [21]:
#knn

knn_clf = KNeighborsClassifier(n_neighbors=9)
knn_clf.fit(X_train,y_train)

y_pred = knn_clf.predict(X_test)

accuracy_knn=round(accuracy_score(y_test,y_pred),3)
print(accuracy_knn)

0.978


Metode Naive Bayes

In [22]:
# naive bayes

gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
YY_pred = gaussian.predict(X_test)
accuracy_nb=round(accuracy_score(y_test,YY_pred)* 100, 2)
print(accuracy_nb)

100.0


In [23]:
# buat model knn

import pickle

with open('knnB_pickle','wb') as r:
    pickle.dump(knn_clf,r)

In [24]:
#buat model naive bayes
import pickle

with open('nb1_pickle','wb') as r:
    pickle.dump(gaussian,r)